In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.3 MB/s eta 0:00:00


In [ ]:
# !pip install reportlab diffusers transformers accelerate

import pandas as pd
import random
import torch
import re
import urllib.parse
from diffusers import StableDiffusionPipeline
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors

# -----------------------------
# 1. Configuration & AI Setup
# -----------------------------

device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    if device == "cuda":
        pipe.enable_attention_slicing()
except Exception as e:
    print(f"SD Load Warning: {e}")
    pipe = None

# -----------------------------
# 2. Helper Functions
# -----------------------------

def normalize(text):
    return re.sub(r'[^a-zA-Z ]', '', str(text).lower()).strip()

def ingredient_match_score(recipe_ingredients, user_ingredients):
    recipe_set = set(normalize(i) for i in recipe_ingredients.split(","))
    user_set = set(normalize(i) for i in user_ingredients)
    matched = recipe_set.intersection(user_set)
    score = len(matched) / len(user_set) if user_set else 0
    return score, matched

def get_recipes(df, user_ingredients, diet_type, max_time):
    user_ing_list = [normalize(i) for i in user_ingredients.split(",")]
    results = []

    for _, row in df.iterrows():
        if row["DietType"].lower() != diet_type.lower():
            continue
        if row["CookingTime_Minutes"] > max_time:
            continue

        score, matched = ingredient_match_score(
            row["Ingredients"], user_ing_list
        )

        # Single ingredient → relaxed matching
        if len(user_ing_list) == 1 and score >= 0.2:
            results.append((score, row))

        # Multiple ingredients → stricter matching
        elif len(user_ing_list) > 1 and score >= 0.6:
            results.append((score, row))

    results.sort(key=lambda x: x[0], reverse=True)

    if not results:
        return pd.DataFrame()

    return pd.DataFrame([r[1] for r in results])

def generate_cooking_steps_ai(recipe):
    ingredients = [i.strip() for i in recipe["Ingredients"].split(",")]
    main_ing = ", ".join(ingredients[:3])

    steps = [
        f"Prepare {main_ing} by washing and chopping as needed.",
        "Preheat your pan or oven to the correct temperature.",
        f"Cook {main_ing} gently, maintaining flavor and texture.",
        "Combine all ingredients and mix well.",
        f"Season appropriately for a {recipe['DietType']} dish.",
        f"Serve {recipe['RecipeName']} hot and fresh."
    ]

    random.shuffle(steps)
    return steps[:5]

def calculate_nutrition(recipe, user_servings):
    base = recipe.get("Estimated_People_Served", 1)
    scale = user_servings / base

    protein = recipe.get("Protein_g", 0) * scale
    carbs = recipe.get("Carbs_g", 0) * scale
    fat = recipe.get("Fat_g", 0) * scale

    calories = (protein * 4) + (carbs * 4) + (fat * 9)

    return {
        "protein": round(protein, 1),
        "carbs": round(carbs, 1),
        "fat": round(fat, 1),
        "calories": round(calories, 1)
    }

def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

def generate_image(recipe):
    if pipe is None:
        return None

    prompt = f"Professional food photography of {recipe['RecipeName']} with {recipe['Ingredients']}"
    image = pipe(prompt, guidance_scale=7.5).images[0]

    filename = f"{sanitize_filename(recipe['RecipeName'])}.png"
    image.save(filename)
    return filename

def get_youtube_link(recipe_name):
    query = urllib.parse.quote(recipe_name + " recipe")
    return f"https://www.youtube.com/results?search_query={query}"

# -----------------------------
# 3. PDF Generation
# -----------------------------

def save_recipe_to_pdf(recipe, nutrition, image_path, output_pdf="recipe_output.pdf"):
    c = canvas.Canvas(output_pdf, pagesize=A4)
    width, height = A4
    y = height - 50

    c.setFont("Helvetica-Bold", 26)
    c.setFillColor(colors.HexColor("#2E86C1"))
    c.drawString(50, y, recipe["RecipeName"])
    y -= 40

    c.setFont("Helvetica-Bold", 12)
    c.setFillColor(colors.darkgreen)
    c.drawString(
        50, y,
        f"Calories: {nutrition['calories']} kcal | "
        f"Protein: {nutrition['protein']}g | "
        f"Carbs: {nutrition['carbs']}g | "
        f"Fat: {nutrition['fat']}g"
    )
    y -= 30

    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, y, "Ingredients")
    y -= 20

    c.setFont("Helvetica", 11)
    for ing in recipe["Ingredients"].split(","):
        c.drawString(70, y, f"• {ing.strip()}")
        y -= 14

    y -= 10
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, y, "Cooking Steps")
    y -= 20

    steps = generate_cooking_steps_ai(recipe)
    c.setFont("Helvetica", 11)
    for i, step in enumerate(steps, 1):
        c.drawString(70, y, f"{i}. {step}")
        y -= 16

    if image_path:
        img = Image.open(image_path)
        aspect = img.height / img.width
        img_w = width - 100
        img_h = img_w * aspect
        if y - img_h < 50:
            c.showPage()
            y = height - 50
        c.drawImage(ImageReader(image_path), 50, y - img_h, img_w, img_h)
        y -= img_h + 20

    yt = get_youtube_link(recipe["RecipeName"])
    c.setFont("Helvetica-Bold", 10)
    c.setFillColor(colors.red)
    c.drawString(50, y, yt)
    c.linkURL(yt, (50, y, width - 50, y + 10))

    c.save()

# -----------------------------
# 4. Main Program
# -----------------------------

def main():
    df = pd.read_excel("/content/recipes_with_ingredient_quantities (1).xlsx")

    print("🍽️ AI Recipe Generator Ready")

    u_ing = input("Enter ingredients (comma separated): ")
    u_diet = input(f"Diet type {df['DietType'].unique()}: ")
    u_time = int(input("Max cooking time (mins): "))
    u_servings = int(input("Number of servings: "))

    recipes = get_recipes(df, u_ing, u_diet, u_time)

    if recipes.empty:
        print("❌ No recipes found")
        return

    print(f"\n🔥 Found {len(recipes)} recipes\n")

    for i, recipe in recipes.iterrows():
        nutrition = calculate_nutrition(recipe, u_servings)

        print("=" * 50)
        print(f"🍲 {recipe['RecipeName']}")
        print(f"👥 Servings: {u_servings}")
        print(f"⏱ Time: {recipe['CookingTime_Minutes']} mins")
        print(f"🔥 Calories: {nutrition['calories']} kcal")
        print(f"💪 Protein: {nutrition['protein']} g")
        print(f"🍞 Carbs: {nutrition['carbs']} g")
        print(f"🧈 Fat: {nutrition['fat']} g")
        print(f"🛒 Ingredients: {recipe['Ingredients']}")
        print(f"🎥 {get_youtube_link(recipe['RecipeName'])}")

    top_recipe = recipes.iloc[0]
    nutrition = calculate_nutrition(top_recipe, u_servings)
    img = generate_image(top_recipe)
    save_recipe_to_pdf(top_recipe, nutrition, img)

    print("\n✅ PDF + Image Generated Successfully")

if __name__ == "__main__":
    main()


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


🍽️ AI Recipe Generator Ready
Enter ingredients (comma separated): Chicken
Diet type ['Non-Vegetarian' 'Vegetarian']: Non-Vegetarian
Max cooking time (mins): 15
Number of servings: 2
❌ No recipes found
